<a href="https://colab.research.google.com/github/YNNJN/colorPOOL_Data/blob/master/%EC%98%88%EC%B8%A1%ED%96%89%EB%A0%AC%EA%B3%84%EC%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. 과정 요약
- 예측행렬 계산 : Sparse matrix의 결측값 비율을 낮추기 위해 진행
    - 그룹에 대한 예측 수행 → 전체 행렬로 확장  
    1) 테마 간 유사도 분석을 통한 클러스터링  
    2) 행렬 인수분해를 통한 색채-테마 관계 예측  
    3) 색채-테마 관계 예측 행렬 정규화 및 보정  

In [ ]:
# connect to google drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
theme_df = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/quantizeTheme.csv')
color_df = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/quantize.csv')
matrixData = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/referenceMatrix.csv')

## 1. 비슷한 색채를 포함하는 테마 그룹핑
- 계산되지 않은 테마(값이 0) 중 가장 평점이 높은 테마를 기준으로 유사도 계산
    - 코사인 계수 활용
    - 유사도가 높은 순서대로 테마를 추가하며 값을 예측
    - 이후 존재하는 기준 값과의 오차를 계산
    - 오차 값이 급격히 커지는 테마 직전까지만을 해당 테마 그룹에 추가

In [ ]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

def cos_sim(A, B):
    normA = norm(A)
    normB = norm(B)
    if normA * normB == 0: # rounding error 이슈 해결을 위한 조건문
        return 0
    else:
        return dot(A, B)/(normA * normB)

In [ ]:
cols_len = len(matrixData.loc[0])
m_columns = matrixData.columns

sim_matrix = pd.DataFrame(columns=m_columns, dtype=float)

for r in range(len(matrixData)): 
    max = 0
    max_index = -1
    for c in range(cols_len):
        rating = matrixData.iloc[r][c]
        if rating:
            if max < rating:
                max = rating
                max_index = c
  
    kj = matrixData.iloc[max_index]
    arr = [0] * cols_len

    for i in range(cols_len):
        if i != max_index:
            ki = matrixData.iloc[i]
            arr[i] = cos_sim(ki, kj)
        else:
            arr[i] = 1.0

    sim_matrix = sim_matrix.append(pd.Series(arr, index=m_columns), ignore_index=True)

sim_matrix

In [ ]:
sim_matrix.to_csv('gdrive/My Drive/colorPOOL/kulerData/simMatrix.csv')

In [ ]:
# 테마 그룹 리스트